This module manages brings together all the classes for extracting time-series at multiple plot locations
 
author:  Dr Milto Miltiadou
version: 1.0

In [ ]:
import sys

# check if GEE is already imported to avoid requesting authenticatiation multiple times
modulename = 'ee'
if modulename not in sys.modules: 
   # import GEE and Authenticate, token or log in will be asked from web browser
   import ee
   ee.Authenticate()
   ee.Initialize()
#else:
#   print('GEE already imported')
   # google earth engine already imported and authenticated


modulename = 'ipynb_masks'
if modulename not in sys.modules:
    %run Masks.ipynb
    sys.modules['ipynb_masks'] = None
#else
    # module already loaded

modulename = 'ipynb_Utils'
if modulename not in sys.modules:
    import Utils
    # adding an identifier to sys.modules to avoiding loading the same file multiple times
    sys.modules['ipynb_Utils'] = None 
#else
   # Utils modules has already been loaded somewhere else

# check if GEE is already imported to avoid requesting authenticatiation multiple times
modulename = 'ipynb_Sentinel1'
if modulename not in sys.modules:
    %run Sentinel1.ipynb
    # adding an identifier to sys.modules to avoiding loading the same file multiple times
    sys.modules['ipynb_Sentinel1'] = None 
#else
   # Sentinel1 modules has already been loaded somewhere else

modulename = 'ipynb_Sentinel2b'
if modulename not in sys.modules:
    %run Sentinel2b.ipynb
    # adding an identifier to sys.modules to avoiding loading the same file multiple times
    sys.modules['ipynb_MapVisualisation'] = None 
#else
   # Utils modules has already been loaded somewhere else

import sys
modulename = 'ipynb_FieldData'
if modulename not in sys.modules:
    %run FieldData.ipynb
    # adding an identifier to sys.modules to avoiding loading the same file multiple times
    sys.modules['FieldData'] = None 
#else
   # Utils modules has already been loaded somewhere else

# check if GEE is already imported to avoid requesting authenticatiation multiple times
modulename = 'Masks'
if modulename not in sys.modules:
    # import Masks Class
    %run Masks.ipynb
# else:
    # Mask class already imported and authenticated




In [ ]:
import pandas as pd
import numbers
from datetime import date

class PlotToSat:
## @brief method that sets the year of interest
    def setYear(self,year):
        # check each collection to see if year exist and set valid to false if year doesn't exist
        self.year = year
        startdate = str(self.year)+'-01-01'
        enddate   = str(self.year)+'-12-31'
        today = date.today()
        cy = today.year
        if (cy==year): 
            enddate = today.strftime("%Y-%m-%d")
        # S1
        ly = int(self.sentinel1Info["launchDate"   ][0:4])
        ry = int(self.sentinel1Info["retirmentDate"][0:4])
        if (year<ly or year>ry):
            print("WARNING: Sentinel-1 data will not be exported as the year requested is before launched date or after retirement date.")
            self.sentinel1Info["startDate"]=""
            self.sentinel1Info["endDate"  ]=""
        elif (year==ly):
            self.sentinel1Info["startDate"]=self.sentinel1Info["launchDate"]
            self.sentinel1Info["endDate"]=enddate
        elif (year==ry):
           self.sentinel1Info["startDate"]=startdate
           self.sentinel1Info["endDate"]=self.sentinel1Info["retirmentDate"]
        else :
            self.sentinel1Info["startDate"]=startdate
            self.sentinel1Info["endDate"]=enddate
        # S2
        ly = int(self.sentinel2Info["launchDate"][0:4])
        if (year<ly or year>ry):
            print("WARNING: Sentinel-2 data will not be exported as the year requested is before launched date.")
            self.sentinel2Info["startDate"]=""
            self.sentinel2Info["endDate"  ]=""
        elif (year==ly):
            self.sentinel2Info["startDate"]=self.sentinel2Info["launchDate"]
            self.sentinel2Info["endDate"]=enddate
        elif (year==ry):
           self.sentinel2Info["startDate"]=startdate
           self.sentinel2Info["endDate"]=self.sentinel2Info["retirmentDate"]
        else :
            self.sentinel2Info["startDate"]=startdate
            self.sentinel2Info["endDate"]=enddate

        print ("Sentinel1: ",self.sentinel1Info["startDate"] ,self.sentinel1Info["endDate"])
        print ("Sentinel2: ",self.sentinel2Info["startDate"] ,self.sentinel2Info["endDate"])




    ## Method that adds the field data to the Manager
    # 
    # @param[in] properties: a dictionary containing the name of the csv file containing the 
    #   plot iformation, the projection and 
    #   the name of the column that give the year that the field data were collected
    #   e.g. {'csvfilename':csvFile,'proj':"EPSG:3042", labelOfyearCol':"year"}

    def addFieldData(self,properties): # csvfile,proj,yearlabel):

        self.csvDF = FieldData(properties)
        self.maxSize = self.csvDF.getLen()



    def __init__(self,geometry,fieldDataProperties,year):
        self.samplingSize = 400
        self.sampleCount  = 0
        self.maxSize      = 0
        self.csvDF        = None
        self.year         = 2017
        self.filedataFilewithIDs = "fieldDataWithIdentifiers/FieldDataWithIdentifiers.csv"
        #----------------------------------------------------
        # Initialise collection 
        #----------------------------------------------------
        todayDate = date.today().strftime("%Y-%m-%d")
        #----------------------------------------------------
        self.sentinel1Info = {
            "identifier": 'sentinel-1',
            "collection": "COPERNICUS/S1_GRD",
            "launchDate" : "2014-04-01",
            "retirmentDate":todayDate, #not retired yet
            "startDate"   : "",
            "endDate"     : "",
            "added"     : False,
            "aspects"   : False
        }       
        #----------------------------------------------------
        self.sentinel2Info = {
            "identifier": 'sentinel-2',                                              
            "collection": "COPERNICUS/S2_SR_HARMONIZED",
            "launchDate" : "2015-05-23",
            "retirmentDate":todayDate, #not retired yet
            "startDate"   : "",
            "endDate"     : "",
            "added"     : False,
            "clouds"    : 50
        } 



        self.numberOfAvailableCols = 2
        # the labels of the available collections
        self.listOfAvailableCollections = ['sentinel-1', 'sentinel-2']
        self.listOfAvailCollections = [sentinel1Info,sentinel2Info]
        # else list have the smae  size and it is correct
        self.geometry  = geometry

        self.fiedplotData = None
        self.proj = "EPSG:3042"
        self.masks = {}
        self.year = 0
        self.radius = 30
        self.csvfile = None
        self.bufferredPoints = None
        self.colX = "CX"
        self.colY = "CY"
        if not ('outPlotFileWithIDs' in fieldDataProperties):
            fieldDataProperties['outPlotFileWithIDs'] = self.filedataFilewithIDs
        # else:
          # outFlotFileWithIDs has been defined by the user  
        self.addFieldData(fieldDataProperties)
        self.setYear(year)
        if(self.csvDF == None or self.maxSize==0):
            raise Exception("Manager: Plot data loaded are not correct!")
       
        
            

    ## @brief method that prints the collections that are available within the system
    def printAvailableCollections(self):
        # new approach
        print("There are ", self.numberOfAvailableCols," collections available within the system:")
        
        labels      = [self.sentinel1Info["identifier"],
                       self.sentinel2Info["identifier"]]
        
        collections = [self.sentinel1Info["collection"],
                       self.sentinel2Info["collection"]]
        
        data = {"label": labels, "collection":collections}
        df = pd.DataFrame(data)
        print(df)
            
            

    
    ## Method that adjust the sampling of the plot data to avoid processing all of them at once and crusing
    #  newSampling the the sampling limit
    def setSampling(self, newSampling):
        self.samplingSize = newSampling
        print("NEW SAMPLING: = ", self.samplingSize)


    

    ## @brief method that sets the masks that will be applied 
    #  @param[in] newMasks the masks to be used
    def setMasks(self, newMasks):
        self.masks = newMasks






    ## method that adds a collection to the manager
    # @param[in] label the label of the collection to be added e.g., 'sentinel-1'
    # @param[in] a variable that may be used if needed e.g., cloudpercentage in sentinel2
    def addCollection(self,label, vari):
        if(label=='sentinel-1'):
            self.sentinel1Info["added"]  = True
            if(isinstance(vari,bool)):
                self.sentinel1Info["aspects"]  = vari
            else :
                print("WARNING: Variable imported along with sentinel-1 is not a Boolean (True or False) value - Aspect maps will not be applied")
            print("Sentinel 1 added")
        elif (label == 'sentinel-2'):
            self.sentinel2Info["added"]  = True
            if(isinstance(vari,numbers.Number) and vari>0 and vari<100):
                self.sentinel2Info["clouds"] = vari
            else :
                print("WARNINGING: Cloud value should be a number between (0,100). A 50 cloud coverage will be used.")
                self.sentinel2Info["clouds"] = 50
            print("Sentinel 2 added")
            
    #loop through each collection and export data 

    def exportCollections(self,startOffeaturesFilenamesWithSampling,gdrivefolder):
        if(self.sentinel1Info["startDate"]!="" and 
        self.sentinel1Info["endDate"]!="" and 
        self.sentinel1Info["added"]):
            print("Sentinel 1 collection parameters loaded are valid and system is now intepreting data")
            s1 = Sentinel1(
                self.geometry,
                self.sentinel1Info["startDate"],
                self.sentinel1Info["endDate"],
                self.masks,
                self.sentinel1Info["aspects"]
                )
            lys = int(self.sentinel1Info["startDate" ][0:4])
            lye = int(self.sentinel1Info["endDate"   ][0:4])
            if (lys!=lye):
                raise Exception("ERROR: start end end data should have been in the same year")
            else: 
                s1.byMonth(lys)
                if(self.sentinel1Info["aspects"]):
                    s1bandsAsc = s1.getCollectionToBandsAsc()
                    s1bandsDes = s1.getCollectionToBandsDes()
                    self.csvDF.exportFeaturesMeanStdCSV(s1bandsAsc,startOffeaturesFilenamesWithSampling+"_S1Asc",gdrivefolder)
                    self.csvDF.exportFeaturesMeanStdCSV(s1bandsDes,startOffeaturesFilenamesWithSampling+"_S1Des",gdrivefolder)
                else :
                   s1bands = s1.getCollectionToBands()            
                   self.csvDF.exportFeaturesMeanStdCSV(s1bands,startOffeaturesFilenamesWithSampling+"_S1",gdrivefolder)
        else:
            print("WARNING: Sentinel 1 collection parameters loaded are NOT valid and system is skipping this collection")


        if(self.sentinel2Info["startDate"]!="" and 
        self.sentinel2Info["endDate"]!="" and 
        self.sentinel2Info["added"]):
            print("Sentinel 2 collection parameters loaded are valid and system is now intepreting data")
            s2 = Sentinel2(
                self.geometry,
                str(self.sentinel2Info["startDate"]),
                str(self.sentinel2Info["endDate"]),
                self.sentinel2Info["clouds"],
                self.masks
                )
            lys = int(self.sentinel2Info["startDate" ][0:4])
            lye = int(self.sentinel2Info["endDate"   ][0:4])
            if (lys!=lye):
                raise Exception("ERROR: start end end data should have been in the same year")
            else: 
                s2.byMonth(lys)
                s2bands = s2.getCollectionToBands()
                self.csvDF.exportFeaturesMeanStdCSV(s2bands,startOffeaturesFilenamesWithSampling+"_S2",gdrivefolder)
        else:
                print("WARNING: Sentinel 2 collection parameters loaded are NOT valid and system is skipping this collection")

    def exprtFeaturesMinMax(self,gdrivefolder,startOffeaturesFilename, min,max):
        currentMin = min
        currentMax = max
        strCurrentMin = str(currentMin  ).rjust(10,'0')
        strCurrentMax = str(currentMax-1).rjust(10,'0')
        startOffeaturesFilenamesWithSampling = startOffeaturesFilename+"_"+strCurrentMin+"_"+strCurrentMax
        bufferredPoints = self.csvDF.createBufferedPoints(currentMin,currentMax)
        if bufferredPoints == None :
            print ("WARNING: exprtFeaturesMinMax: No data exported because range provided does not exist.")
            return
        
        self.exportCollections(startOffeaturesFilenamesWithSampling,gdrivefolder)
        
     
    ## method that exports a set of files containing the feature vectors
    def exportFeatures(self,gdrivefolder,startOffeaturesFilename):
        if (self.sentinel1Info["added"] == False and self.sentinel2Info["added"] == False):
            raise Exception("ERROR: No collections have been added!")
        # else at least one collection has been 

        
        lenOfFieldData = self.csvDF.getLen()
        currentMin = 0
        currentMax = self.samplingSize
        while currentMin<lenOfFieldData :
            strCurrentMin = str(currentMin  ).rjust(10,'0')
            strCurrentMax = str(currentMax-1).rjust(10,'0')
            startOffeaturesFilenamesWithSampling = startOffeaturesFilename+"_"+strCurrentMin+"_"+strCurrentMax
            bufferredPoints = self.csvDF.createBufferedPoints(currentMin,currentMax)
            if bufferredPoints == None :
                continue
            
            self.exportCollections(startOffeaturesFilenamesWithSampling,gdrivefolder)
        
            currentMin = currentMax
            currentMax = currentMax + self.samplingSize
        print("    *** End of exportation = Check progress on the Tasks tab at GEE ***")
            


                




